In [240]:
import lsst.daf.butler as daf_butler
import numpy as np
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import astropy.units as u
afwDisplay.setDefaultBackend("firefly")

In [241]:
collections=["u/elhoward/TOO/0725J","u/rea3/test_0725J_v10/20250729T163637Z","u/elhoward/TOO/0725J/MMADS_candidates","LSSTCam/runs/nightlyValidation/20250728/w_2025_30/DM-50157","u/elhoward/TOO/0725J/DESGW_candidates_v2"]
repo="embargo_new"

In [249]:
MyBut = daf_butler.Butler(repo,collections=collections)

In [243]:
finalVisits = []
for k in np.arange(2025072800099,2025072800159):
    if int(str(k)[-1])!=9:
        finalVisits.append(k)

In [250]:
detector = 11
visit = 2025072800101

In [251]:
# MyBut.registry.queryCollections("*ightly*20250728*")

In [252]:
diffRefs = list(MyBut.query_datasets("difference_image",where=f"exposure={visit} and instrument='LSSTCam' and detector={detector}"))
tempRefs = list(MyBut.query_datasets("template_matched",where=f"exposure={visit} and instrument='LSSTCam' and detector={detector}"))
searchRefs = list(MyBut.query_datasets("preliminary_visit_image",where=f"exposure={visit} and instrument='LSSTCam' and detector={detector}"))

In [253]:
diff = MyBut.get(diffRefs[0])
temp = MyBut.get(tempRefs[0])
search = MyBut.get(searchRefs[0])

In [189]:
display = afwDisplay.Display(backend="firefly",frame=0)
display.image(search)

In [190]:
display = afwDisplay.Display(backend="firefly",frame=1)
display.image(temp)

In [187]:
display = afwDisplay.Display(backend="firefly",frame=2)
display.image(diff)

In [ ]:
# The below is covered in the FindingCandidates notebook

In [ ]:
# searchRefs = list(MyBut.query_datasets("preliminary_visit_image",where=f"exposure={visit} and instrument='LSSTCam'"))

MMADs_candidates = [[231.072434,-37.622634],
                    [235.548892,-35.002876],
                    [234.538140,-36.254760]]

for r in searchRefs:
    exp = MyBut.get(r)
    for coord in MMADs_candidates:
        try:
            if exp.containsSkyCoords(coord[0]*u.deg, coord[1]*u.deg)[0]:
                print(r,coord)
        except:
            print("Error encountered")

In [217]:
myTup = tuple([int(str(x)) for x in tuple(DESGW_visits)])

In [234]:
AllPointings = tuple([int(str(x)) for x in tuple(finalVisits[0:9])])

In [193]:
DESGW_visits = []
for v in finalVisits:
    if int(str(v)[-1]) in (4,5,7,8):
        DESGW_visits.append(v)

In [ ]:
searchRefs = list(MyBut.query_datasets("preliminary_visit_image",where=f"exposure in {AllPointings} and instrument='LSSTCam'"))

candidates = {
"AT2025srk":[234.755524,-36.425993],
"AT2025srl":[234.90473,-36.726032],
"AT2025srm":[232.987656,-36.816564],
"AT2025shz":[231.072434,-37.622634],
"AT2025sib":[235.548892,-35.002876],
"AT2025sia":[234.538140,-36.254760]}

for name,coord in zip(candidates.keys(),candidates.values()):
    for r in searchRefs:
        exp = MyBut.get(r)
        try:
            if exp.containsSkyCoords(coord[0]*u.deg, coord[1]*u.deg)[0]:
                print(name,r.dataId["visit"],r.dataId["detector"])
        except:
            print("Error encountered with coord:{}, visit:{}".format(coord,r.dataId["visit"]))

AT2025srk 2025072800105 25
Error encountered with coord:[234.755524, -36.425993], visit:2025072800100
Error encountered with coord:[234.755524, -36.425993], visit:2025072800101
Error encountered with coord:[234.755524, -36.425993], visit:2025072800102
Error encountered with coord:[234.755524, -36.425993], visit:2025072800103
Error encountered with coord:[234.755524, -36.425993], visit:2025072800104
Error encountered with coord:[234.755524, -36.425993], visit:2025072800105
Error encountered with coord:[234.755524, -36.425993], visit:2025072800106
Error encountered with coord:[234.755524, -36.425993], visit:2025072800107
Error encountered with coord:[234.755524, -36.425993], visit:2025072800108
AT2025srk 2025072800104 170
Error encountered with coord:[234.755524, -36.425993], visit:2025072800100
Error encountered with coord:[234.755524, -36.425993], visit:2025072800108
Error encountered with coord:[234.755524, -36.425993], visit:2025072800100
Error encountered with coord:[234.755524, -36

In [264]:
AllPointings

(2025072800100,
 2025072800101,
 2025072800102,
 2025072800103,
 2025072800104,
 2025072800105,
 2025072800106,
 2025072800107,
 2025072800108)

In [ ]:
2025072800101, 